## Before starting this tutorial
- Install DFTTK: https://github.com/PhasesResearchLab/dfttk
- Install YPHON: https://github.com/PhasesResearchLab/YPHON 
- Follow the POTCAR setup for pymatgen: https://pymatgen.org/installation.html#potcar-setup

## Import neccessary libraries

In [ ]:
# Enable automatic reloading of modules
%reload_ext autoreload
%autoreload 2

# Standard Library Imports
import os
import subprocess

# Third-Party Library Imports
import numpy as np
import plotly.graph_objects as go

# DFTTK Imports
from dfttk.configuration import Configuration
from dfttk.plotly_format import plot_format

## Convergence tests

Create a folder to run the volume relaxation. We will use the relaxed structure for our convergence tests.

In [ ]:
path = "conv_test/volume_relax"
os.makedirs(path, exist_ok=True)

Copy the `POSCAR` file to the volume relaxation folder.

In [ ]:
subprocess.run(["cp", "POSCAR", path])

Create and configure the Configuration object. 

In [ ]:
vasp_cmd = ["mpirun", "/opt/packages/VASP/VASP6/6.4.3/ONEAPI/vasp_std"]
config_Al = Configuration(path, "config_Al", vasp_cmd)

Configure the `job script` for volume relaxation. This example is based on the Bridges-2 supercomputer at the Pittsburgh Supercomputing Center (PSC).

In [ ]:
# Write the job script to the config_Al.path directory
job_script = """#!/bin/bash
#SBATCH --job-name=Al
#SBATCH -A dmr190011p
#SBATCH -p RM
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH -t 1:00:00
#SBATCH -o vasp.out
#SBATCH -e vasp.err

ulimit -s unlimited
export OMP_NUM_THREADS=1
echo "SLURM_NTASKS: " $SLURM_NTASKS
module purge
module load intel-oneapi
module load hdf5/1.12.0-intel20.4
module load anaconda3
conda activate dfttk
module load VASP/6.4.3-intel
python run_dfttk.py
echo "Current SLURM JobID: $SLURM_JOB_ID"
sacct --job=$SLURM_JOB_ID
"""

with open(os.path.join(config_Al.path, "job.sh"), "w") as f:
    f.write(job_script)

🚀 Run the volume relaxation.

In [ ]:
config_Al.run_volume_relax(
    material_type="metal",
    encut=520,
    kppa=4000,
)

⏳ After the job has finished, copy the relaxed `CONTCAR` file to the directory above. We will use this for the convergence tests.

In [ ]:
subprocess.run(["cp", "CONTCAR", "../POSCAR"], cwd=path)

Update the configuration path.

In [ ]:
config_Al.path = "conv_test"

Write the `job script` for the convergence tests.

In [ ]:
# Write the job script to the config_Al.path directory
job_script = """#!/bin/bash
#SBATCH --job-name=Al
#SBATCH -A dmr190011p
#SBATCH -p RM
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH -t 1:00:00
#SBATCH -o vasp.out
#SBATCH -e vasp.err

ulimit -s unlimited
export OMP_NUM_THREADS=1
echo "SLURM_NTASKS: " $SLURM_NTASKS
module purge
module load intel-oneapi
module load hdf5/1.12.0-intel20.4
module load anaconda3
conda activate dfttk
module load VASP/6.4.3-intel
python run_dfttk.py
echo "Current SLURM JobID: $SLURM_JOB_ID"
sacct --job=$SLURM_JOB_ID
"""

with open(os.path.join(config_Al.path, "job.sh"), "w") as f:
    f.write(job_script)

🚀 Run the convergence tests.

In [ ]:
config_Al.run_conv_test()

⏳ After the jobs have finished, analyze the convergence of the total energy with respect to the cutoff energy (ENCUT).

In [ ]:
encut_conv_df, fig = config_Al.analyze_encut_conv()
encut_conv_df

Analyze the convergence of the total energy with respect to the k-point mesh per reciprocal atom (KPPA).

In [ ]:
kpoints_conv_df, fig = config_Al.analyze_kpoints_conv()
kpoints_conv_df

Based on the results, we select `ENCUT = 520 eV` and `kppa = 4000`, as the energy difference compared to the previous values is below 1 meV/atom, ensuring sufficient accuracy.

## Energy-volume curve

Create a folder called config_Al.

In [ ]:
os.makedirs("config_Al", exist_ok=True)

Copy the `POSCAR` file to the config_Al folder.

In [ ]:
subprocess.run(["cp", "POSCAR", "config_Al"])

Define the path to the config_Al folder and create a Configuration object.

In [ ]:
path = "config_Al"
vasp_cmd = ["mpirun", "/opt/packages/VASP/VASP6/6.4.3/ONEAPI/vasp_std"]
config_Al = Configuration(path, "Al", vasp_cmd)

Write the `job script` for the energy-volume curve jobs.

In [ ]:
# Write the job script to the config_Al.path directory
job_script = """#!/bin/bash
#SBATCH --job-name=Al
#SBATCH -A dmr190011p
#SBATCH -p RM-shared
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH -t 1:00:00
#SBATCH -o vasp.out
#SBATCH -e vasp.err

ulimit -s unlimited
export OMP_NUM_THREADS=1
echo "SLURM_NTASKS: " $SLURM_NTASKS
module purge
module load intel-oneapi
module load hdf5/1.12.0-intel20.4
module load anaconda3
conda activate dfttk
module load VASP/6.4.3-intel
python run_dfttk.py
echo "Current SLURM JobID: $SLURM_JOB_ID"
sacct --job=$SLURM_JOB_ID
"""

with open(os.path.join(config_Al.path, "job.sh"), "w") as f:
    f.write(job_script)

Configure the energy-volume settings

In [ ]:
volumes = [74, 72, 70, 68, 66, 64, 62, 60]
material_type = "metal"
encut = 520
kppa = 4001
config_Al.ev_curve_settings(material_type=material_type, volumes=volumes, encut=encut, kppa=kppa, copy_magmom=False)

🚀 Run the energy-volume curve calculations for the chosen volumes.

In [ ]:
config_Al.run_ev_curves()

⏳ After the job has finished, process the results of the energy-volume curves.

In [ ]:
config_Al.process_ev_curve(eos_name="BM4")

Plot the energy-volume curve.

In [ ]:
fig = config_Al.ev_curve.plot()
fig.show()

Examine and analyze the EOS parameters.

In [ ]:
config_Al.ev_curve.eos_parameters

## Phonons

Re-write the `job script` for the phonon calculations if needed.

In [ ]:
# Write the job script to the config_Al.path directory
job_script = """#!/bin/bash
#SBATCH --job-name=Al
#SBATCH -A dmr190011p
#SBATCH -p RM-shared
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH -t 5:00:00
#SBATCH -o vasp.out
#SBATCH -e vasp.err

ulimit -s unlimited
export OMP_NUM_THREADS=1
echo "SLURM_NTASKS: " $SLURM_NTASKS
module purge
module load intel-oneapi
module load hdf5/1.12.0-intel20.4
module load anaconda3
conda activate dfttk
module load VASP/6.4.3-intel
echo "Current SLURM JobID: $SLURM_JOB_ID"
sacct --job=$SLURM_JOB_ID
"""

with open(os.path.join(config_Al.path, "job.sh"), "w") as f:
    f.write(job_script)

Configure the phonon settings.

In [ ]:
phonon_volumes = [74, 72, 70, 68, 66, 64, 62, 60]
scaling_matrix = ((2, 0, 0), (0, 2, 0), (0, 0, 2))
kppa = 4001
relax = False

config_Al.phonons_settings(phonon_volumes=phonon_volumes, kppa=kppa, scaling_matrix=scaling_matrix, relax=relax)

🚀 Run the phonon calculations in parallel.

In [ ]:
config_Al.run_phonons()

⏳ After the jobs have finished, generate the phonon DOS using YPHON in each phonon_folder and store all the results in YPHON_results.

In [ ]:
config_Al.generate_phonon_dos()

Using the phonon DOS, calculate the harmonic properties.

In [ ]:
number_of_atoms = 4
volumes_fit = np.linspace(0.98*60, 1.02*74, 1000)
temperatures = np.arange(0, 1010, 10)
config_Al.process_phonons(number_of_atoms=number_of_atoms, volumes_fit=volumes_fit, temperatures=temperatures)

Plot both the original and scaled phonon DOS. The scaled phonon DOS is adjusted to the number of atoms, N, that you specify. YPHON scales the area under the phonon DOS curve to 3N.

In [ ]:
config_Al.phonons.plot_scaled_dos(number_of_atoms=4)

Plot the scaled phonon DOS for multiple volumes together.

In [ ]:
config_Al.phonons.plot_multiple_dos()

Plot the harmonic properties.

In [ ]:
# Plot helmholtz_energy, entropy, or heat_capacity
fig_phonons_t, fig_phonons_v = config_Al.phonons.plot_harmonic("helmholtz_energy")
fig_phonons_t.show()
fig_phonons_v.show()

## Debye-Grüneisen model

Calculate the vibrational contribution to the free energy, entropy, and heat capacity using the Debye model.

In [ ]:
volumes = np.linspace(0.98*60, 1.02*74, 1000)
temperatures = np.arange(0, 1010, 10)
config_Al.process_debye(volumes=volumes, temperatures=temperatures, scaling_factor=0.617, gruneisen_x=2/3)

Plot the Debye properties.

In [ ]:
# Plot helmholtz_energy, entropy, or heat_capacity
fig_debye_t, fig_debye_v = config_Al.debye.plot("helmholtz_energy")
fig_debye_t.show()
fig_debye_v.show()

## Thermal electronic contribution

Re-write the `job script` for the thermal electronic calculations if needed.

In [ ]:
# Write the job script to the config_Al.path directory
job_script = """#!/bin/bash
#SBATCH --job-name=Al
#SBATCH -A dmr190011p
#SBATCH -p RM-shared
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH -t 5:00:00
#SBATCH -o vasp.out
#SBATCH -e vasp.err

ulimit -s unlimited
export OMP_NUM_THREADS=1
echo "SLURM_NTASKS: " $SLURM_NTASKS
module purge
module load intel-oneapi
module load hdf5/1.12.0-intel20.4
module load anaconda3
conda activate dfttk
module load VASP/6.4.3-intel
echo "Current SLURM JobID: $SLURM_JOB_ID"
sacct --job=$SLURM_JOB_ID
"""

with open(os.path.join(config_Al.path, "job.sh"), "w") as f:
    f.write(job_script)

Configure the thermal electronic settings.

In [ ]:
volumes = [74, 74, 72, 70, 68, 66, 64, 62, 60]
scaling_matrix = ((1, 0, 0), (0, 1, 0), (0, 0, 1))
kppa = 4001
config_Al.thermal_electronic_settings(volumes=volumes, kppa=kppa, scaling_matrix=scaling_matrix)

🚀 Run the thermal electronic jobs in parallel.

In [ ]:
config_Al.run_thermal_electronic()

⏳ After the jobs have finished, calculate the thermal electronic properties.

In [ ]:
volumes_fit = np.linspace(0.98*60, 1.02*74, 1000)
temperatures = np.arange(0, 1010, 10)
config_Al.process_thermal_electronic(volumes_fit, temperatures)

Plot the thermal electronic properties.

In [ ]:
# Plot helmholtz_energy, entropy, or heat_capacity
fig_thermal_electronic_t = config_Al.thermal_electronic.plot_vt("heat_capacity_vs_temperature")
fig_thermal_electronic_v = config_Al.thermal_electronic.plot_vt("heat_capacity_vs_volume")
fig_thermal_electronic_t.show()
fig_thermal_electronic_v.show()

## Quasiharmonic Approximation

Calculate the quasi-harmonic properties using multiple methods: debye, debye_thermal_electronic, phonons, and phonons_thermal_electronic.

In [ ]:
config_Al.process_qha("debye", P = 0)
config_Al.process_qha("debye_thermal_electronic", P = 0)
config_Al.process_qha("phonons", P = 0)
config_Al.process_qha("phonons_thermal_electronic", P = 0)

In [ ]:
# Plot helmholtz_energy_pv, volume, CTE, LCTE, entropy, heat_capacity, enthalpy, bulk_modulus, gibbs_energy
fig_qha = config_Al.qha.plot("phonons_thermal_electronic", P = 0, plot_type="helmholtz_energy_pv")
fig_qha.show()

## Comparison with Experiments

In [ ]:
def add_trace(fig, x, y, mode, name, color, dash=None, symbol=None):
    """Helper function to add a trace to the figure."""
    trace = go.Scatter(
        x=x,
        y=y,
        mode=mode,
        name=name,
        line=dict(color=color, dash=dash) if dash else dict(color=color),
        marker=dict(symbol=symbol, color=color) if symbol else None,
    )
    fig.add_trace(trace)

C<sub>p</sub> vs. T

In [ ]:
# Initialize figure
fig = go.Figure()
temperature = config_Al.qha.temperatures

# Ref: https://janaf.nist.gov/tables/Al-001.html
T_janaf = np.array([0, 100, 200, 298.15, 300, 400, 500, 600, 700, 800, 900])  # K
Cp_janaf = np.array([0, 12.997, 21.338, 24.209, 24.247, 25.784, 26.842, 27.886, 29.100, 30.562, 32.308])  # J/mol/K
add_trace(fig, T_janaf, Cp_janaf, mode="markers", name="JANAF", color="black", symbol="circle-open")

# Data from calculations
methods = {
    "Ph+El": {
        "data": config_Al.qha.methods["phonons_thermal_electronic"]['0.00_GPa']["Cp"],
        "color": "#EF553B",
        "dash": "dash",
    },
    "Ph": {
        "data": config_Al.qha.methods["phonons"]['0.00_GPa']["Cp"],
        "color": "#636EFA",
    },
    "Debye+El": {
        "data": config_Al.qha.methods["debye_thermal_electronic"]['0.00_GPa']["Cp"],
        "color": "#AB63FA",
        "dash": "dash",
    },
    "Debye": {
        "data": config_Al.qha.methods["debye"]['0.00_GPa']["Cp"],
        "color": "#00CC96",
    },
}

# Add traces for each method
for name, props in methods.items():
    Cp = props["data"] * 1.60218e-19 / 4 * 6.022e23  # Convert to J/mol/K
    add_trace(fig, temperature, Cp, mode="lines", name=name, color=props["color"], dash=props.get("dash"))

# Format and display the plot
plot_format(fig, xtitle="Temperature (K)", ytitle="C<sub>p</sub> (J/mol/K)", width=650, height=600)
fig.update_xaxes(range=[0, 930], tick0=0, dtick=100)
fig.update_yaxes(range=[0, 35], tick0=0, dtick=5)
fig.show()

CTE vs. T

In [ ]:
# Initialize figure
fig = go.Figure()
temperature = config_Al.qha.temperatures

# Ref: https://iopscience.iop.org/article/10.1088/0959-5309/53/3/305
T_C_Wilson = np.array([0, 100, 200, 300, 400, 500, 600, 650]) # °C
T_K_Wilson = T_C_Wilson + 273.15  # Convert to Kelvin
CTE_linear = np.array([22, 25.4, 26.5, 27.8, 29.9, 32.5, 35.5, 37.2]) # Linear CTE
CTE_volume = CTE_linear * 3 # Convert to volumetric CTE
add_trace(fig, T_K_Wilson, CTE_volume, mode="markers", name="Wilson (1941)", color="black", symbol="circle-open")

# Data from calculations
methods = {
    "Ph+El": {
        "data": config_Al.qha.methods["phonons_thermal_electronic"]['0.00_GPa']["CTE"],
        "color": "#EF553B",
        "dash": "dash",
    },
    "Ph": {
        "data": config_Al.qha.methods["phonons"]['0.00_GPa']["CTE"],
        "color": "#636EFA",
    },
    "Debye+El": {
        "data": config_Al.qha.methods["debye_thermal_electronic"]['0.00_GPa']["CTE"],
        "color": "#AB63FA",
        "dash": "dash",
    },
    "Debye": {
        "data": config_Al.qha.methods["debye"]['0.00_GPa']["CTE"],
        "color": "#00CC96",
    },
}

# Add traces for each method
for name, props in methods.items():
    add_trace(fig, temperature, props["data"], mode="lines", name=name, color=props["color"], dash=props.get("dash"))

# Format and display the plot
plot_format(fig, xtitle="Temperature (K)", ytitle="CTE (10<sup>-6</sup> K<sup>-1</sup>)", width=650, height=600)
fig.update_xaxes(range=[0, 930], tick0=0, dtick=100)
fig.update_yaxes(range=[0, 120], tick0=0, dtick=20)
fig.show()

In [ ]:
experiments = {
    "Janaf": {
        "ref": "https://janaf.nist.gov/tables/Al-001.html",
        "temperatures": T_janaf.tolist(),
        "pressure": 0,
        "heatCapacity": Cp_janaf.tolist(),
    },
    "Wilson (1941)": {
        "ref": "https://iopscience.iop.org/article/10.1088/0959-5309/53/3/305",
        "temperatures": T_K_Wilson.tolist(),
        "pressure": 0,
        "heatCapacity": CTE_volume.tolist(),
    }
}

config_Al.add_experiments(experiments)

## MongoDB

In [ ]:
config_Al.add_metadata(vasp_version = "6.4.3", comment = "FCC Al 4-atom supercell")

In [ ]:
# Enter the details of your MongoDB database
connection_string = "enter_connection_string"
db_name = "enter_db_name"
collection_name = "enter_collection_name"
document = config_Al.to_mongodb(connection_string, db_name, collection_name)